In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
import numpy as np
import sys
import os
import random
from collections import defaultdict
import statistics
sys.path.append("../")
from utils import plotly_utils

### Study html writes on sampled 20 URLs from carta

In [4]:
data = json.load(open('../datacollect/data/carta_urls_100.json', 'r'))
data = random.sample(data, 20)
json.dump(data, open('test_urls.json', 'w+'), indent=2)

#### Number of writes per page

In [11]:
dirs = os.listdir('writes')
number_writes = defaultdict(list)
for dirr in dirs:
    for i in range(5):
        writes = json.load(open(f'writes/{dirr}/{i}.json', 'r'))
        number_writes[dirr].append(len(writes))
num_writes = {k: (min(v),  statistics.median(v), max(v)) for k, v in number_writes.items()}
num_writes

{'soulisauctions_com': (26, 26, 26),
 'niadart_org': (59, 129, 129),
 'alllightexpanded_com': (42597, 42597, 42597),
 'www_myragreene_com': (0, 0, 0),
 'www_bonhams_com': (416, 416, 416),
 'www_wallergallery_com': (46, 46, 46),
 'www_arprim_org': (116, 116, 116),
 '127garazh_org': (84, 84, 84),
 'chicagoimagists_com': (28, 28, 28),
 'www_masonfineartandevents_com': (24, 24, 24),
 '2022_vbexhibitions_hk': (40, 40, 40),
 'paulwongprojects_com': (45, 45, 45),
 'www_visualartscentre_ca': (40, 54, 54),
 'greekpavilion2022_gr': (11, 11, 11),
 'nationalmuseumofmexicanart_org': (43, 54, 54),
 'www_artbrussels_com': (291, 291, 291),
 'www_gkb-furniture_com': (94, 94, 94),
 'www_chloebeaulac_com': (13, 13, 13),
 'wyld_gallery': (143, 143, 143),
 'canadianart_ca': (66, 74, 74)}

#### Order of each element in the writes

In [4]:
dirs = os.listdir('writes')
stats = {'total_element': 0, 'allpresent_element': 0}
total_empty_xpath = 0
max_order_diffs = []
for dirr in dirs:
    max_order_diff = []
    xpath_order = defaultdict(list)
    for i in range(5):
        writes = json.load(open(f'writes/{dirr}/{i}.json', 'r'))
        for j, write in enumerate(writes):
            if len(write['xpath']) == 0:
                total_empty_xpath += 1
                continue
            tup = (write['xpath'][-1], write['method'], write['arg'])
            xpath_order[tup].append(j)
    for xpath, orders in xpath_order.items():
        num_orders = len(orders)
        stats['total_element'] += 1
        if len(orders) == 5:
            stats['allpresent_element'] += 1
        if len(orders) > 2:
            max_order, min_order = max(orders), min(orders)
            max_order_diff.append(max_order - min_order)
    max_order_diff.sort()
    if len(max_order_diff) > 0:
        ninety_percentile = int(len(max_order_diff) * 0.9)
        ninety_percentile = max_order_diff[ninety_percentile]
        fifty_percentile = int(len(max_order_diff) * 0.5)
        fifty_percentile = max_order_diff[fifty_percentile]
        hundred_percentile = max_order_diff[-1]
        max_order_diffs.append({'50th': fifty_percentile, '90th': ninety_percentile, '100th': hundred_percentile})
            
# stats, total_empty_xpath
# df = pd.DataFrame({1: max_order_diffs})
# plotly_utils.plot_CDF(df)
df = pd.DataFrame(max_order_diffs)
plotly_utils.plot_bar(df)

#### Intersection of all the xpaths across 5 runs

In [47]:
# Get the longest prefix of two lists contains the same set of elements
def get_longest_prefix(l1, l2):
    l1_set, l2_set = set(l1), set(l2)
    if l1_set.issubset(l2_set) or l2_set.issubset(l1_set):
        return min(len(l1), len(l2))
    l1_unique, l2_unique = l1_set - l2_set, l2_set - l1_set
    intersection = set(l1).intersection(set(l2))
    max_prefix = 0
    l1_prefix, l2_prefix = set(), set()
    for i in range(min(len(l1), len(l2))):
        if l1[i] in l1_unique or l2[i] in l2_unique:
            break
        l1_prefix.add(l1[i])
        l2_prefix.add(l2[i])
        if l1_prefix.issubset(intersection) and l2_prefix.issubset(intersection):
            if l1_prefix == l2_prefix:
                max_prefix = i+1
    return max_prefix

# get_longest_prefix(['a', 'd', 'b', 'c'], ['a', 'b', 'c', 'e', 'a'])

In [48]:
dirr = 'chicagoimagists_com'
idx = [0, 4]
w1 = json.load(open(f'writes/{dirr}/{idx[0]}.json', 'r'))
w2 = json.load(open(f'writes/{dirr}/{idx[1]}.json', 'r'))
new_w1, new_w2 = [], []
for j, write in enumerate(w1):
    if len(write['xpath']) == 0:
        total_empty_xpath += 1
        continue
    tup = (write['xpath'][-1], write['method'], write['arg'])
    new_w1.append(tup)
for j, write in enumerate(w2):
    if len(write['xpath']) == 0:
        total_empty_xpath += 1
        continue
    tup = (write['xpath'][-1], write['method'], write['arg'])
    new_w2.append(tup)
# print(json.dumps(new_w1, indent=2), json.dumps(new_w2, indent=2))
get_longest_prefix(new_w1, new_w2)

25

In [51]:
dirs = os.listdir('writes')
longest_prefix_info = []
for dirr in dirs:
    print(dirr)
    all_writers = []
    for i in range(5):
        writes = json.load(open(f'writes/{dirr}/{i}.json', 'r'))
        new_writers = []
        for j, write in enumerate(writes):
            if len(write['xpath']) == 0:
                total_empty_xpath += 1
                continue
            tup = (write['xpath'][-1], write['method'], write['arg'])
            new_writers.append(tup)
        all_writers.append(new_writers)
    max_prefix = get_longest_prefix(all_writers[0], all_writers[1])
    for i in range(2, 5):
        max_prefix = get_longest_prefix(all_writers[0][:max_prefix], all_writers[i])
    longest_prefix_info.append({
        'max_prefix': max_prefix, 
        'min_length':  min([len(w) for w in all_writers]),
        'max_length':  max([len(w) for w in all_writers]),
    })
df = pd.DataFrame(longest_prefix_info)
df

soulisauctions_com
niadart_org
alllightexpanded_com
www_myragreene_com
www_bonhams_com
www_wallergallery_com
www_arprim_org
127garazh_org
chicagoimagists_com
www_masonfineartandevents_com
2022_vbexhibitions_hk
paulwongprojects_com
www_visualartscentre_ca
greekpavilion2022_gr
nationalmuseumofmexicanart_org
www_artbrussels_com
www_gkb-furniture_com
www_chloebeaulac_com
wyld_gallery
canadianart_ca


,max_prefix,min_length,max_length
0,26,26,26
1,129,129,129
2,42597,42597,42597
3,0,0,0
4,39,416,416
5,46,46,46
6,116,116,116
7,84,84,84
8,28,28,28
9,24,24,24
